In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Load Preprocessed data

In [2]:
# load data after the processing in part A
df=pd.read_csv("preprocessed_data.csv")
df.head()

C:\Users\gerasimos\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (6,11,12,14,15) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,InvoiceAmount,AmountCategory,InvoiceDate,InvoiceYear,EnteredDate,Days_to_Enter,SupplierRef,PaidDate,Days_to_Pay,PaidYear,PaymentStatus,DocumentType,PaymentType,ModifiedDate,UserID,PostedDate,Invoicecurrency,ConfirmedRisk,Client,InvoiceType
0,1.78,<100,2016-12-18 00:00:00.000,2016.0,2016-03-10 00:00:00.000,NaN,ONE007,2016-06-07 00:00:00.000,NaN,2016.0,F,PI,PB,2021-07-16 13:20:05.177,SB24,2016-03-11 00:00:00.000,GBP,0.0,A,Invoice
1,50.23,<100,2019-03-23 00:00:00.000,2019.0,2016-04-04 00:00:00.000,NaN,XMA001,2016-06-01 00:00:00.000,NaN,2016.0,F,PI,PB,2021-07-16 13:20:05.177,DT61,2016-04-04 00:00:00.000,GBP,NaN,A,Invoice
2,639.00,100-1k,2016-12-22 00:00:00.000,2016.0,2016-04-26 00:00:00.000,NaN,LAN080,2016-07-12 00:00:00.000,NaN,2016.0,F,PI,PB,2021-07-16 13:20:05.177,SB24,2016-05-03 00:00:00.000,GBP,NaN,A,Invoice
3,1045.00,1k-10k,2016-12-22 00:00:00.000,2016.0,2016-04-26 00:00:00.000,NaN,LAN080,2016-07-12 00:00:00.000,NaN,2016.0,F,PI,PB,2021-07-16 13:20:05.177,SB24,2016-05-03 00:00:00.000,GBP,NaN,A,Invoice
4,33.73,<100,2019-06-22 00:00:00.000,2019.0,2016-07-14 00:00:00.000,NaN,DAW002,2016-09-06 00:00:00.000,NaN,2016.0,F,PI,PB,2021-07-16 13:20:05.177,DT61,2016-07-18 00:00:00.000,GBP,NaN,A,Invoice


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3644635 entries, 0 to 3644634
Data columns (total 20 columns):
 #   Column           Dtype  
---  ------           -----  
 0   InvoiceAmount    float64
 1   AmountCategory   object 
 2   InvoiceDate      object 
 3   InvoiceYear      float64
 4   EnteredDate      object 
 5   Days_to_Enter    float64
 6   SupplierRef      object 
 7   PaidDate         object 
 8   Days_to_Pay      float64
 9   PaidYear         float64
 10  PaymentStatus    object 
 11  DocumentType     object 
 12  PaymentType      object 
 13  ModifiedDate     object 
 14  UserID           object 
 15  PostedDate       object 
 16  Invoicecurrency  object 
 17  ConfirmedRisk    float64
 18  Client           object 
 19  InvoiceType      object 
dtypes: float64(6), object(14)
memory usage: 556.1+ MB


# Deal with missing values

In [4]:
# the entries with NaN values in ConfirmedRisk have to be deleted 
# since they don't provide information for the class of the entry

df = df[df["ConfirmedRisk"].isnull()==False]

In [5]:
# drop columns that cannot be used in modelling

df.drop(["InvoiceDate", "EnteredDate", "PaidDate", "ModifiedDate", "PostedDate", "SupplierRef"],axis=1, inplace=True)

In [6]:
# check what to do with UserID

len(df["UserID"].unique())

124

In [7]:
# could one-hot-encode UserID but would increase dramatically the dimensiolatiy of the data 
# without adding significant value (at least intuitively)
#drop UserID column

df.drop("UserID",axis=1, inplace=True)

In [8]:
df.head()

,InvoiceAmount,AmountCategory,InvoiceYear,Days_to_Enter,Days_to_Pay,PaidYear,PaymentStatus,DocumentType,PaymentType,Invoicecurrency,ConfirmedRisk,Client,InvoiceType
0,1.78,<100,2016.0,NaN,NaN,2016.0,F,PI,PB,GBP,0.0,A,Invoice
7,77.22,<100,2016.0,NaN,19.0,2016.0,F,PI,PB,GBP,0.0,A,Invoice
8,154.44,100-1k,2016.0,NaN,19.0,2016.0,F,PI,PB,GBP,0.0,A,Invoice
9,300.00,100-1k,2016.0,0.0,8.0,2016.0,F,PI,PB,GBP,0.0,A,Invoice
10,500.00,100-1k,2016.0,0.0,8.0,2016.0,F,PI,PB,GBP,0.0,A,Invoice


In [9]:
#check for missing values
total = df.shape[0]

cols = list(df.columns)

for column in cols:
    print(f"{column} missing values: \t{df[column].isnull().sum()} ({round(df[column].isnull().sum()/total*100,2)}%)")

InvoiceAmount missing values: 	0 (0.0%)
AmountCategory missing values: 	0 (0.0%)
InvoiceYear missing values: 	2 (0.0%)
Days_to_Enter missing values: 	4266 (0.39%)
Days_to_Pay missing values: 	811431 (74.15%)
PaidYear missing values: 	810933 (74.11%)
PaymentStatus missing values: 	3990 (0.36%)
DocumentType missing values: 	57859 (5.29%)
PaymentType missing values: 	890979 (81.42%)
Invoicecurrency missing values: 	0 (0.0%)
ConfirmedRisk missing values: 	0 (0.0%)
Client missing values: 	0 (0.0%)
InvoiceType missing values: 	0 (0.0%)


In [10]:
#Drop columns with more than 70% missing values
df.drop(["Days_to_Pay", "PaidYear", "PaymentType"], axis=1, inplace=True)


#only a small proportion of numerical values are missin. Fill with the mean value
df["InvoiceYear"].fillna(df["InvoiceYear"].mean(), inplace=True)
df["Days_to_Enter"].fillna(df["Days_to_Enter"].mean(), inplace=True)

#Fill with "Other" missing categorical values with less than 10% missing data
df["PaymentStatus"].fillna("Other", inplace=True)
df["DocumentType"].fillna("Other", inplace=True)

# One-Hot-Encoding

In [11]:
df = pd.get_dummies(data=df)

In [12]:
df.head()

,InvoiceAmount,InvoiceYear,Days_to_Enter,ConfirmedRisk,AmountCategory_100-1k,AmountCategory_10k-100k,AmountCategory_1k-10k,AmountCategory_<100,AmountCategory_>100k,PaymentStatus_#,...,Invoicecurrency_EUR,Invoicecurrency_GBP,Invoicecurrency_NOK,Invoicecurrency_PLN,Invoicecurrency_USD,Client_A,Client_B,Client_C,InvoiceType_Credit,InvoiceType_Invoice
0,1.78,2016.0,20.952042,0.0,0,0,0,1,0,0,...,0,1,0,0,0,1,0,0,0,1
7,77.22,2016.0,20.952042,0.0,0,0,0,1,0,0,...,0,1,0,0,0,1,0,0,0,1
8,154.44,2016.0,20.952042,0.0,1,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,1
9,300.00,2016.0,0.000000,0.0,1,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,1
10,500.00,2016.0,0.000000,0.0,1,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,1


# Normalize data

In [13]:
from sklearn.preprocessing import MinMaxScaler


df[list(df.columns)] = MinMaxScaler().fit_transform(df[list(df.columns)])

# Class Balance

In [14]:
X = df.drop("ConfirmedRisk", axis=1)
y = df["ConfirmedRisk"]

In [15]:
from collections import Counter

print (f"Class Balance {Counter(y)}")

Class Balance Counter({0.0: 1090212, 1.0: 4056})


The problem suffers from extremly imbalanced classes. SMOTE algorithm can be used to tackle the problem.

In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.5, random_state=42, stratify=y)

print(f"Class balance in training data: \t {Counter(y_train)}")

Class balance in training data: 	 Counter({0.0: 545106, 1.0: 2028})


In [17]:
from imblearn.over_sampling import BorderlineSMOTE

bsmote = BorderlineSMOTE(sampling_strategy="minority", random_state=42)


X_bsm_train, y_bsm_train = bsmote.fit_resample(X_train, y_train)


print (f"Class balance in training data after BorderlineSmote Oversampling: \t {Counter(y_bsm_train)}")

Class balance in training data after BorderlineSmote Oversampling: 	 Counter({0.0: 545106, 1.0: 545106})


# Logistic Regression

In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score

clf = LogisticRegression()
clf.fit(X_bsm_train, y_bsm_train)

y_preds = clf.predict(X_test)

print(confusion_matrix(y_test, y_preds))
print(classification_report(y_test, y_preds))

print(f"AUROC: {roc_auc_score(y_test, y_preds)}")

C:\Users\gerasimos\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[[520873  24233]
 [   440   1588]]
              precision    recall  f1-score   support

         0.0       1.00      0.96      0.98    545106
         1.0       0.06      0.78      0.11      2028

    accuracy                           0.95    547134
   macro avg       0.53      0.87      0.55    547134
weighted avg       1.00      0.95      0.97    547134

AUROC: 0.8692909507834283


# ------------------------------------------------------------------------------------------------------------

# Future Work

- Test the performance of the model on each one of the three clients
- Try to train 3 different models (one for each client) instead of one model for all of them
- Try other models (SVM, Random Forest, XGBoost, LightGBM, ...)
- Could try Feature Selection & Hyperparameter Tuning
- Try XAI tools to increase explainability of the models